# Genome plotter

This ipython script was written to create a visual representation of the human genome, reflecting its chemical composition (GC content), distribution of protein coding genes, and location of know genome wide association signals.

### Requirements:

* Previously downloaded and processed genomic data (prepared by `prepare_data.sh`): chunked chromosomes in bed format, gwas catalog, gencode file.
* non-standard python libraries: [pandas](http://pandas.pydata.org/), [numpy](http://www.numpy.org/), [cairosvg](http://cairosvg.org/), [pybedtools](https://pythonhosted.org/pybedtools/)


### About the data:

In this work I was using the GRCH38 build of the human genome (Ensembl release 83), GENCODE version , the GWAS catalog was downloaded on 2015.06.16 (genomic coordinates also in GRCH37), the downloadable GWAS catalog was extended with an in-house maintained positive control collection. 


### Further readings and references:

* Human genome:

* GWAS:

### Data sources:

* [Ensembl](Ensembl.org): the European genome database, where all known genomic information is aggregated including genes, transcripts, variations, regulatory elements and many more. They provide access to the human genome via ftp server.

* [GWAS catalog](https://www.ebi.ac.uk/gwas/): manually curated collection of variations in the human genome with known phenotypical changes including [breast size](https://www.ebi.ac.uk/gwas/search?query=Breast%20size) and [polytical ideology](https://www.ebi.ac.uk/gwas/search?query=Political%20ideology). 

* [GENCODE](http://www.gencodegenes.org/): ultimate resource of annotated genetic elements example genes, transcripts, exons etc. 

In [1]:
import time
print "Last modified:", (time.strftime("%d/%m/%Y"))

Last modified: 11/04/2016


## Stage 1.

In the first stage we further process the sequence data, and integrate GENCODE and GWAS data into a single dataframe and save it for plotting.

### Steps:

1. Reading bedfiles with the chunk numbers and the GC content.
2. Run bedtools to find out which chunks overlap with protein coding genes and GWAS hits.
3. Based on the GC content and the genetic overlapping, a color is assigned to each chunk.
4. Constructing a data frame with the above mentioned data.
5. Save dataframe in binary file (pickle).

In [3]:
'''
Importig libraries:
'''

import gzip # For reading data
import pybedtools # For finding overlap between our chunks and genes and GWAS signals
import pandas # data handling
import numpy as np # Working with large arrays
import colorsys # Generate color gradient.
import cairosvg # Converting svg to image
import pickle # Saving dataframes to disk
import os.path # checking if the datafies are already there.

In [31]:
'''
Files used for the anaysis. Let's check if they are really there:
'''

workingDir = os.getcwd()
Chromosome_file_loc = workingDir + "/data/Processed_chr%s.bed.gz"
GWAS_file_loc = workingDir + "/data/processed_GWAS.bed.gz"
GENCODE_file_loc = workingDir + "/data/processed_GENCODE.bed.gz"
outputDir = workingDir + "/processed_data/dataframe_chr%s.pkl"

if not os.path.isfile(GWAS_file_loc):
    print "GWAS file is missing. Run `prepare_data.sh` first!"
if not os.path.isfile(GENCODE_file_loc):
    print "GENCODE file is missing. Run `prepare_data.sh` first!"
if not os.path.isfile(Chromosome_file_loc % 11):
    print "Processed chromosome files are missing! Run `prepare_data.sh` first!"

In [29]:
'''
Functions for generating nice colors and gradients
'''
def linear_gradient(start_hex, finish_hex="#FFFFFF", n=10):
    ''' 
    returns a gradient list of (n) colors between
    two hex colors. start_hex and finish_hex
    should be the full six-digit color string,
    inlcuding the sharp sign (eg "#FFFFFF") 
    '''
    # Starting and ending colors in RGB form
    s = hex_to_RGB(start_hex)
    f = hex_to_RGB(finish_hex)
    
    # Initilize a list of the output colors with the starting color
    RGB_list = [start_hex]
    
    # Calcuate a color at each evenly spaced value of t from 1 to n
    for t in range(1, n):
    
        # Interpolate RGB vector for color at the current value of t
        curr_vector = [
            int(s[j] + (float(t)/(n-1))*(f[j]-s[j]))
            for j in range(3)
        ]

        # Add it to our list of output colors
        RGB_list.append(RGB_to_hex(curr_vector))

    return RGB_list

def hex_to_RGB(hex):
    ''' "#FFFFFF" -> [255,255,255] '''
    # Pass 16 to the integer function for change of base
    return [int(hex[i:i+2], 16) for i in range(1,6,2)]


def RGB_to_hex(RGB):
    ''' [255,255,255] -> "#FFFFFF" '''
    # Components need to be integers for hex to make sense
    RGB = [int(x) for x in RGB]
    return "#"+"".join(["0{0:x}".format(v) if v < 16 else
            "{0:x}".format(v) for v in RGB])

def get_color(row, colors):
    '''
    Based on the values in the submitted row, this
    function picks a color from the color dictionary, and returns it.
    '''
    
    # Built-in parameters:
    threshold = 0.7
    max_diff_value = 0.15

    # At first we get the index:
    try:
        index = int(float(row["GC_content"])*20)
    except:
        index = 0
    
    # Extracting color based on the gene and the index:
    try:
        color = colors[row["Genes"]][index]
    except:
        color = colors[0][index]
        
    # Ok, we have the color, now based on the column we make it a bit darker:
    if row["Column_frac"] > threshold:
        diff = (row["Column_frac"] - threshold)/(1 - threshold)
        factor = 1 - max_diff_value*diff 
        
        # Get rgb code of the hexa code:
        rgb_code = hex_to_RGB(color)
        
        # Get the hls code of the rgb:
        hls_code = colorsys.rgb_to_hls(rgb_code[0]/float(255), 
                                       rgb_code[1]/float(255), 
                                       rgb_code[2]/float(255))
        
        # Get the modifed rgb code:
        new_rgb = colorsys.hls_to_rgb(hls_code[0], 
                                      hls_code[1]*factor, 
                                      hls_code[2])
        
        # Get the modifed hexacode:
        color = RGB_to_hex([new_rgb[0] * 255,
                           new_rgb[1] * 255,
                           new_rgb[2] * 255])
        
    return color  

def def_chromosome_colors(chromosome):
    '''
    This function retruns with an array of colors based on the chromosome number.
    input: chromosome number from 0 to 23 
    output: [[ten colors for intergenic], [ten colors for genic region]]
    
    (Ten is the step number of the gradient showing GC content in the given chunk.)
    '''

    # The colors will be picked from the following array:
    color_set = [
        ["#b74242", "#b79e42"],
        ["#b75f42", "#b3b742"],
        ["#b77c42", "#96b742"],
        ["#b79a42", "#78b742"],
        ["#b7b742", "#5bb742"],
        ["#9ab742", "#42b746"],
        ["#7cb742", "#42b763"],
        ["#5fb742", "#42b780"],
        ["#42b742", "#42b79e"],
        ["#42b75f", "#42b3b7"],
        ["#42b77c", "#4296b7"],
        ["#42b79a", "#4278b7"],
        ["#42b7b7", "#425bb7"],
        ["#429ab7", "#4642b7"],
        ["#427cb7", "#6342b7"],
        ["#425fb7", "#8042b7"],
        ["#4242b7", "#9e42b7"],
        ["#5f42b7", "#b742b3"],
        ["#7c42b7", "#b74296"],
        ["#9a42b7", "#b74278"],
        ["#b742b7", "#b7425b"],
        ["#b7429a", "#b74642"],
        ["#b7427c", "#b76342"],
        ["#b7425f", "#b78042"]
    ]

    # 
    colors = {}
    colors[0] = linear_gradient(color_set[chromosome - 1][0], n=20) # For non genes.
    colors[1] = linear_gradient(color_set[chromosome - 1][1], n=20) # For regions overlapping with genes

    return colors

In [35]:
'''Function for process data'''

def ProcessChromosome(chromosome, Max_rows = 600):
    '''
    This function reads processed genomic data saved in bed format, comressed with gzip.
    
    Input is just the chromosome number and a number how many rows we want to 
    slice the chromosomes (this parameter is optional, default value is 600). 
    
    Output is a dataframe with the following columns:
        "Chunk_no": integer, 
        "GC_content": float (0..1),
        "Genes": integer 1 or 0 -> 1 indicates the chunk overlaps with a gene
        "GWAS": integer 1 or 0 -> 1 indicate overlap with a GWAS signal
        "Colors": hexadecimal RGB code representing the given chunk.
    '''
    
    # The name of the chromosome is X and Y for number 23 and 24 respectively:
    chr_name = chromosome
    if chr_name == 23: chr_name = "X"
    if chr_name == 24: chr_name = "Y"

    # Importing global variables:
    global Chromosome_file_loc
    global GWAS_file_loc
    global GENCODE_file_loc

    # Opening bedfile, read GC contents and chunk number:
    chunk_no = []
    CG_content = []
    with gzip.open(Chromosome_file_loc % chr_name,'r') as bedfile:
        for line in bedfile:
            line = line.strip()
            (chrom, start, end, GC, chunk) = line.split("\t")

            chunk_no.append(chunk)
            CG_content.append(GC)
            
    # As soon as the GC contents are read, we check 
    # if the chunks are overlapping with genes or GWAS hits:

    # Run intersectBed query:
    data_file = pybedtools.BedTool((Chromosome_file_loc % chr_name))
    GENCODE_file = pybedtools.BedTool(GENCODE_file_loc)
    GWAS_file = pybedtools.BedTool(GWAS_file_loc)

    # Get intersecting genes:
    GencodeIntersect = data_file.intersect(GENCODE_file, wa = True)

    # Get intersecting GWAS signals:
    GWASIntersect = data_file.intersect(GWAS_file, wa = True)
    
    # Now we have to loop through all intersecting values:
    Genes = np.zeros(len(CG_content), dtype=np.int)
    GWAS = np.zeros(len(CG_content), dtype=np.int)
    chunks_in_genes = [hit.fields[4] for hit in GencodeIntersect]
    chunks_in_gwas  = [hit.fields[4] for hit in GWASIntersect]
    
    # elements of the arrays corresponding to overlapping chunks
    # will be updated to 1 from 0:
    for index in chunks_in_genes:
        Genes[int(index)] = 1

    for index in chunks_in_gwas:
        GWAS[int(index)] = 1

    # Data organized into dataframe:
    df = pandas.DataFrame({
            "Chunk_no": chunk_no,
            "GC_content": CG_content,
            "Genes": Genes,
            "GWAS": GWAS
        })    
        
    return df


def color_chromosome(df, chromosome, Max_rows = 600):
    '''
    Once we have the dataframe of the chunks we can assign colors based on the 
    chromosome number, CG content, gene annotation.
    '''
    # Based on the maximum number of rows, we assign a row location 
    # for each element:
    # Number of lines the whole genome is broken up
    Max_columns = len(df) / Max_rows

    # Calculating the coordinates for each chunk in the final plot, and add to the df:
    rows = [int(x)/Max_columns for x in df.Chunk_no.tolist()]
    column = [int(x) - Max_columns*(int(x)/Max_columns) for x in df.Chunk_no.tolist()]
    df["Row_no"] = pandas.Series(rows)
    df["Column_no"] = pandas.Series(column)
    df["Column_frac"] = [float(x) / Max_columns for x in column]

    # Picking colors for the chromosome
    colors = def_chromosome_colors(chromosome)

    # Assigning colors from the above defined dictionary:
    df["color"] = df.apply(get_color, axis=1, args=(colors,))    

    return df

def plot_chromosome(df, chromosome, filename, rows = 600):
    '''
    Genomic data processed, colors have been assigned to each chunk.
    Now create an svg image, then render it in a png.
    '''
    width = int(len(df)/rows)
    chromosome_name = chromosome
    # Although I decided to focus on the autosomal chromosomes, the script is ready to deal with
    # the sex chromosomes.
    if chromosome_name == 23: chromosome_name = "X"
    if chromosome_name == 24: chromosome_name = "Y"        

    # The size of box which represent a single chunk of the genome:
    box_width = 2
    box_height = 2

    # svg file is saved:
    #f = open('chromosome1.html', 'w')
    plot = '<svg width="%s" height="%s">\n' % (width*1.5*box_width, rows*1.5*box_height)

    # Plotting genome based on GC content and gene annotation:
    for index, color in enumerate(df.color):

        # Get x,y coordinates based on index:
        y = (index / width) * (box_height + 1)
        x = (index - (index / width)*width)*(box_width+1)

        # Generate svg line:
        plot +='<rect x="%s" y="%s" width="%s" height="%s" style="stroke-width:1;stroke:%s; fill: %s" />\n' % (x, y, box_width + 1 , box_height + 1, color, color)

    # looking through gwas annotation:
    for index, gwas in enumerate(df.GWAS):
        if gwas == 1:
            # based on the index of the given line, we get the x,y coordinates:
            y = (index / width) * (box_height + 1) + 1.5
            x = (index - (index / width)*width)*(box_width+1) + 1.5

            # Drawing circle around the defined point:
            plot += '<circle cx="%s" cy="%s" r="3" stroke="%s" stroke-width="1" fill="%s" />\n' % (x, y, "black", "black")

    ###
    ### Drawing legend for the figure.
    ### A box with the name of the chromosome explanation of the colors
    ### Scale bars etc.
    ###

    # Adding a bigger rectangle, for legend:
    legend_x = 15
    legend_y = 15
    legend_width = 250
    legend_height = 220
    plot += '<g>\n'
    plot += '<rect x="%s" y="%s" rx="20" ry="20" width="%s" height="%s" style="fill:white;stroke:black;stroke-width:2" />\n' % (
            legend_x, legend_y, legend_width, legend_height)

    # Adding chromosome number:
    plot += '<text x="140" y="46" text-anchor="middle" font-family="Verdana" font-size="30">chr%s</text>\n' % (chromosome_name)

    # Adding text: (GC content)
    plot += '<text x="95" y="80" text-anchor="middle" font-family="Verdana" font-size="13">GC content</text>\n'

    # For the color section of the legend, we have to generate a color gradient:
    colors = def_chromosome_colors(chromosome)

    # Adding gene colors:
    y = 85
    x = 20
    height = 15
    width = 7
    for index, col in enumerate (reversed(colors[1])):
        x_coord = x+width*int(index)
        plot += '<rect x="%s" y="%s" width="%s" height="%s" style="stroke-width:1;stroke:%s; fill: %s" />\n' %(x_coord, y, width, height, col, col)
    plot += '<text x="%s" y="%s" font-family="Verdana" font-size="17">%s</text>\n' %(x_coord + 15, y + 12, "Genes")

    # Adding background colors:
    y = 105
    for index, col in enumerate (reversed(colors[0])):
        x_coord = x+width*int(index)
        plot += '<rect x="%s" y="%s" width="%s" height="%s" style="stroke-width:1;stroke:%s; fill: %s" />\n' %(x_coord, y, width, height, col, col)
    plot += '<text x="%s" y="%s" font-family="Verdana" font-size="17">%s</text>\n' %(x_coord + 15, y + 12, "Intergenic")

    # Adding scale bars:
    length = 63
    x = 40
    y = 160
    tick = 5

    plot += '<text x="%s" y="%s" font-family="Verdana" text-anchor="middle"  font-size="15">15kbp</text>\n' % (length/2 + x, y - 10)
    plot += '<line x1="%s" y1="%s" x2="%s" y2="%s" style="stroke:black;stroke-width:3" />\n' %(x,y,x + length,y)
    plot += '<line x1="%s" y1="%s" x2="%s" y2="%s" style="stroke:black;stroke-width:3" />\n' %(x,y-tick,x,y+tick)
    plot += '<line x1="%s" y1="%s" x2="%s" y2="%s" style="stroke:black;stroke-width:3" />\n' %(x + length,y-tick,x + length,y+tick)

    # Adding verical units: 42 units for 500kbp
    # Get the length of one row: 
    length_row = len(df[df.Row_no == 3]) * 480
    length_bar = 50
    bar_size_genome = round(length_bar / 2 * length_row/1e6, 2)

    x = 140
    y = 160
    plot += '<text x="%s" y="%s" font-family="Verdana" font-size="15">%sMbp</text>\n' % (
        x + 15, y, bar_size_genome)
    plot += '<line x1="%s" y1="%s" x2="%s" y2="%s" style="stroke:black;stroke-width:3" />\n' % (
        x, y-length_bar/2, x, y + length_bar/2)
    plot += '<line x1="%s" y1="%s" x2="%s" y2="%s" style="stroke:black;stroke-width:3" />\n' % (
        x-tick,y-length_bar/2,x+tick,y-length_bar/2)
    plot += '<line x1="%s" y1="%s" x2="%s" y2="%s" style="stroke:black;stroke-width:3" />\n' % (
        x-tick,y + length_bar/2,x+tick,y + length_bar/2)

    # Adding a circle to the bottom of the legend:
    circle_r = 5
    circle_x = 40 
    circle_y = 210
    plot += '<circle cx="%s" cy="%s" r="%s" stroke="black" stroke-width="1" fill="black" />\n' % (
            circle_x, circle_y, circle_r)
    plot += '<text x="%s" y="%s" font-family="Verdana" font-size="15">%s</text>\n' % (
            circle_x + 20, circle_y + 5, "Known GWAS signals")

    # Closing legend
    plot += '</g>'

    #f.write('</svg>\n</body>\n</html>\n')
    plot += '</svg>\n'
    #f.close()

    fout = open('%s_chr%s.png' % (filename, chromosome),'w')
    cairosvg.svg2png(bytestring=plot,write_to=fout)
    fout.close()

In [36]:
'''
Now we loop through all autosomes and generate a plot.
In the meanwhile, the dataframes are also saved, and upon a future step,
they can be read back withouth calculating them from the scratch.
'''
for chromosome in range(1, 22):
    print "Processing chromosome %s" % (chromosome)

    # Process chromosome data, assign colors, combine into dataframe:
    df = ProcessChromosome(chromosome)

    # Pickling dataframes:
    filename = outputDir % chromosome
    output = open(filename, 'wb')
    pickle.dump(df, output)
    
    # If we jus want ot read df from file:

    # Add colors to datafrome:
    df = color_chromosome(df, chromosome)
    plot_chromosome(df, chromosome, "plot", 600)


Processing chromosome 1
Processing chromosome 2
Processing chromosome 3
Processing chromosome 4
Processing chromosome 5
Processing chromosome 6
Processing chromosome 7
Processing chromosome 8
Processing chromosome 9
Processing chromosome 10
Processing chromosome 11
Processing chromosome 12
Processing chromosome 13
Processing chromosome 14
Processing chromosome 15
Processing chromosome 16
Processing chromosome 17
Processing chromosome 18
Processing chromosome 19
Processing chromosome 20
Processing chromosome 21
